In [ ]:
!git clone https://github.com/islab-ai/AITutorial.git

In [ ]:
# Colab에서 실행할 경우에만!!
import os
os.chdir("/content/AITutorial/RL/")

In [1]:
!pip install gym==0.20.0

In [2]:
!pip install pygame

In [3]:
import numpy as np

from lib.envs.gridworld import GridworldEnv # 상위 폴더내의 lib 폴더 -> env 폴더 -> gridworld.py 파일에서 GridwordlEnv 를 가져옴

In [4]:
env = GridworldEnv() # 강화학습을 위한 환경을 env 변수에 넣음

<h1> Grid World 탐색

** 요약 **
1. env.nS ☞ number of states in the environment. => 16
2. env.nA ☞ number of actions in the environment. => 4
3. env.P[s] ☞ 각 state 에서 모든 action 에 대한 (trainsition prob, next_state, reward, done) 에 대한 정보를 tuple 로 담고있음.

In [5]:
env._render()

x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T


### 1. Space 확인

In [6]:
# env 의 space 는 총 16개 이다.
print("env 의 총 space 갯수:", env.nS)

# 수행 가능한 action 수는 4개이다 (상 하 좌 우)
print("action 의 갯수:", env.nA)

env 의 총 space 갯수: 16
action 의 갯수: 4


In [7]:
# State 확인
for state in range(env.nS):
    print(state)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


### 2. Action 확인하기
- UP = 0
- RIGHT = 1
- DOWN = 2
- LEFT = 3

In [8]:
env._explain_action()

0:UP(위) 

1:RIGHT(오른쪽) 

2:DOWN(아래)  

3:LEFT(왼쪽) 



### 3. State transition probability 확인하기

In [9]:
env.P[1] # 1번 State에 대한 정보

{0: [(1.0, 1, -1.0, False)],
 1: [(1.0, 2, -1.0, False)],
 2: [(1.0, 5, -1.0, False)],
 3: [(1.0, 0, -1.0, True)]}

In [10]:
env.P[1][0] # 1번 State에서 액션 0번을 취할 경우

[(1.0, 1, -1.0, False)]

#### 위에서 보았듯이,
env.P[s] ☞ 각 state 에서 모든 action 에 대한 (trainsition prob, next_state, reward, done) 에 대한 정보를 tuple 로 담고있다.

env.P는 각 state마다 액션(action), 확률(probability), 다음 상태(next state), 보상(reward), 종료(done)<br>
{action: [(probability, nextstate, reward, done)]} 형식으로 정의 되어 있다.  

예를 들어 1번 state 를 살펴보면

즉,  
- 1번 state 에서 위쪽 (0) 으로 이동하면  
100%의 확률로 1번 state로 transition 하고 (결국 제자리), 보상은 -1 이며 게임을 계속한다 (False)

- 1번 state 에서 오른쪽 (1) 으로 이동하면  
100%의 확률로 2번 state로 transition 하고, 보상은 -1 이며 게임을 계속한다 (False)  
- 1번 state 에서 밑 (2) 으로 이동하면  
100%의 확률로 5번 state로 transition 하고, 보상은 -1 이며 게임을 계속한다 (False)  
- 1번 state 에서 왼쪽 (3) 으로 이동하면  
100%의 확률로 0번 state로 transition 하고, 보상은 -1 이며 게임은 종료된다 (True)  
 - 이는 0번 state 가 terminal state 이기 때문이다.

### 4. 초기 Policy 생성하기

- policy 는 0.25 로 모두 초기화
- 윗줄부터 차례대로 State 0~15 에 대한 Policy를 나타낸다
- 16개의 state 에 uniform random policy 가 할당 되어 있다

In [11]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
print(random_policy)

[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]


### 5. 정책 평가(Policy evaluation)
- 현재 policy에 대한 가치함수 찾기

In [12]:
def policy_eval(random_policy, env, discount_factor=1.0, theta=0.00001): 
    # theta 는 interation 을 멈추는 시기를 알려주는 역치
    # 각 state 의 value 값을 0으로 초기화
    V = np.zeros(env.nS)
    turn=1
    while True:
        # 이전 iteration 에 비해 State Value 값이 얼마나 값이 변했는지 확인하기 위해 설정
        delta = 0  
        print("---평가 횟수: ", turn)
        for state in range(env.nS):
            print("현재 state: ", state)
            # env.nS = 16 이므로 state 의 값 0~15 까지 => 모든 state 를 한 번씩 돌게된다
            v = 0 # 초기화
            
            # random uniform policy (0.25로 통일) 를 action 별로 action probability 와 함께 가지고 온다
            for action, action_prob in enumerate(random_policy[state]):
                
                # 반드시 기억해야 할 것, 
                # env.P는 {action: [(probability, nextstate, reward, done)]} 형식으로 정의 되어 있다.
                # env.P[state][action] = 특정 state 에서 특정 actiond에 대한 P, S', R, Done 의 값
                # V[next_state] = 다음 state 의 value
                
                # for ~~~~ in env.P[state][action]: 이므로,
                # 현재 k 에 대해 특정 state 에 대해서 모든 action 별로 하나씩 value 값을 업데이트 한다
                for  transition_prob, next_state, reward, done in env.P[state][action]:
                    print("state: ", state)
                    print("선택한 action: ", action)                    
                    # Value 값 계산
                    
                    # for 문을 이용해서 각 state 마다 서로 다른 action 값에 따른 next state 의 value 값을
                    # 계속 더해주기 위해 (Sigma 를 구현하기 위해) += 을 사용해주도록 한다 
                    
                    v += action_prob * (reward + discount_factor * transition_prob * V[next_state])
            
            # 이전 iteration 에 비해 State Value 값이 얼마나 값이 변했는지 확인 = delta
            delta = max(delta, np.abs(v - V[state]))
            # State 의 value 값 update
            V[state] = v
            print("--------------------")
            turn +=1
        # value 의 변화량이 theta (정해준 역치)보다 적으면 break 
        if delta < theta:
            break
            
    return np.array(V)

### 확인하기 - random policy

In [13]:
# Policy Evaluation 수행
v = policy_eval(random_policy, env)

---평가 횟수:  1
현재 state:  0
state:  0
선택한 action:  0
state:  0
선택한 action:  1
state:  0
선택한 action:  2
state:  0
선택한 action:  3
--------------------
현재 state:  1
state:  1
선택한 action:  0
state:  1
선택한 action:  1
state:  1
선택한 action:  2
state:  1
선택한 action:  3
--------------------
현재 state:  2
state:  2
선택한 action:  0
state:  2
선택한 action:  1
state:  2
선택한 action:  2
state:  2
선택한 action:  3
--------------------
현재 state:  3
state:  3
선택한 action:  0
state:  3
선택한 action:  1
state:  3
선택한 action:  2
state:  3
선택한 action:  3
--------------------
현재 state:  4
state:  4
선택한 action:  0
state:  4
선택한 action:  1
state:  4
선택한 action:  2
state:  4
선택한 action:  3
--------------------
현재 state:  5
state:  5
선택한 action:  0
state:  5
선택한 action:  1
state:  5
선택한 action:  2
state:  5
선택한 action:  3
--------------------
현재 state:  6
state:  6
선택한 action:  0
state:  6
선택한 action:  1
state:  6
선택한 action:  2
state:  6
선택한 action:  3
--------------------
현재 state:  7
state:  7
선택한 action:  0
state:  7
선

In [14]:
print("Value Function:")
print(v)
print("")

print("Value Function reshape:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]

Value Function reshape:
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]

